# 题目

> 有 n 个城市，其中一些彼此相连，另一些没有相连。如果城市 a 与城市 b 直接相连，且城市 b 与城市 c 直接相连，那么城市 a 与城市 c 间接相连。  
省份是一组直接或间接相连的城市，组内不含其他没有相连的城市。  
给你一个 n x n 的矩阵 isConnected ，其中 isConnected[i][j] = 1 表示第 i 个城市和第 j 个城市直接相连，而 isConnected[i][j] = 0 表示二者不直接相连。  
返回矩阵中省份的数量。

# 方法一：并查集

> 计算连通分量数的另一个方法是使用并查集。初始时，每个城市都属于不同的连通分量。遍历矩阵 isConnected ，如果两个城市之间有相连关系，则它们属于同一个连通分量，对它们进行合并。  
遍历矩阵 isConnected 的全部元素之后，计算连通分量的总数，即为省份的总数。

## 复杂度

- 时间复杂度: $O(n^2logn)$ ，其中 $n$ 为城市的数量。

> 需要遍历矩阵 isConnected 中的所有元素，时间复杂度是 $O(n^2)$ ，如果遇到相连关系，则需要进行 2 次查找和最多 1 次合并，一共需要进行 $2n^2$ 次查找和最多 $n^2$ 次合并，因此总时间复杂度是 $O(2n^2log⁡n^2)=O(n^2log⁡n^2)$ 。并查集使用路径压缩，但是没有使用按秩合并，最坏情况下的时间复杂度是 $O(n^2log⁡n^2)$，平均情况下的时间复杂度依然是 $O(n^2\alpha(n))$ ，其中 $\alpha$ 为阿克曼函数的反函数，$\alpha$ 可以认为是一个很小的常数。

- 空间复杂度: $O(n)$ ，其中 $n$ 为城市的数量。

> 需要使用数组 parent 记录每个城市所属的连通分量的祖先。

## 代码

In [1]:
# 构建并查集
class UnionFind:
    def __init__(self, n):
        self.fa = list(range(n))  # 用于记录每个元素的值及其指向的值。第x个位置表示该元素值为x，其对应的值fa[x]表示元素x指向元素fa[x]
        self.rank = [1] * n  # 用于记录某个代表元素对应的集合的复杂度（节点个数）
        self.part = n  # n个元素共有几个集合
    
    # 找到某个集合的代表元素，并进行路径压缩
    def find(self, x):
        
        # 若元素x不指向自己，则寻找x指向的元素fa[x],看其是否指向自己
        # 若fa[fa[x]]仍不指向自己，则继续找，直到找到指向自己的元素（一个集合的代表元素）
        # 找到代表元素后，迭代函数返回该代表元素的值，在回溯过程中，由于最后一个find返回了代表元素的值，因此迭代中的每个fa[x]都会变得指向代表元素
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        
        return self.fa[x]

    # 将两个元素所对应的集合合并
    def union(self, x, y):
        
        # 找到元素x和元素y所在集合的代表元素，并进行路径压缩
        fx, fy = self.find(x), self.find(y)
        
        # 若元素x和元素y所在集合为同一个，返回None
        if fx == fy:
            return False
        
        # 将rank更小的集合连接到rank更大的集合
        # 若元素x所在集合的最大路径长度小于元素x所在集合的最大路径长度，则交换fx和fy
        if self.rank[fx] < self.rank[fy]:
            fx, fy = fy, fx  # fx代表x,y中rank更大的集合的代表元素，fy代表x,y中rank更小的集合的代表元素
        
        # 更新合并后集合的代表元素的rank
        self.rank[fx] += self.rank[fy]
        
        # 将较小集合的代表元素指向较大集合的代表元素
        self.fa[fy] = fx
        self.part -= 1  # 每将单个元素合并一次，总集合数-1
        return True

    # 判断两个元素是否属于同一集合
    def is_connected(self, x, y):
        return self.find(x) == self.find(y)

# 解法
class Solution:
    def findCircleNum(self, isConnected):
        
        cities = len(isConnected)  # 城市总数
        uf = UnionFind(cities)  # 根据城市总数构建并查集
        
        # 遍历所有城市，并查看城市i和城市j之间是否有连通关系
        for i in range(cities):
            for j in range(i + 1, cities):
                # 若城市i和城市j联通，则将i,j放到一个集合
                if isConnected[i][j] == 1:
                    uf.union(i, j)
        
        # fa即为合并后的集合情况
        parent = uf.fa
        
        # 查看parent中共有多少个代表元素（parent[i] == i），其数目等于省份数量
        provinces = sum(parent[i] == i for i in range(cities))
        return provinces

#### 测试一

In [2]:
isConnected = [[1,1,0],[1,1,0],[0,0,1]]

test = Solution()
test.findCircleNum(isConnected)

2

#### 测试二

In [3]:
isConnected = [[1,0,0],[0,1,0],[0,0,1]]

test = Solution()
test.findCircleNum(isConnected)

3